In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import display

import torch

# تحميل البيانات
cols_dict = {
    'ratings': ['user_id', 'movie_id', 'rating', 'timestamp'],
    'users': ['user_id', 'gender', 'age', 'occupation', 'zip_code'],
    'items': ['movie_id', 'title', 'genre']
}
dir = 'C:/Users/lenovo/Desktop/LinUCB-HybridRecommender/data/1m/'

ratings_data = pd.read_csv(dir + 'ratings.dat', sep='::', names=cols_dict['ratings'], engine='python')
users_data = pd.read_csv(dir + 'users.dat', sep='::', names=cols_dict['users'], engine='python')
items_data = pd.read_csv(dir + 'movies.dat', sep='::', names=cols_dict['items'], encoding='latin-1', engine='python')

display(ratings_data.head(3), users_data.head(3), items_data.head(3))
print(ratings_data.shape, users_data.shape, items_data.shape)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


(1000209, 4) (6040, 5) (3883, 3)


In [18]:
# import pandas as pd
# import torch

# # تعريف هيكل البيانات
# cols_dict = {
#     'ratings': ['user_id', 'movie_id', 'rating', 'timestamp'],
#     'users': ['user_id', 'gender', 'age', 'occupation', 'zip_code'],
#     'items': ['movie_id', 'title', 'genre']
# }

# # المسار إلى ملفات البيانات 100K
# dir = 'C:/Users/lenovo/Desktop/LinUCB-HybridRecommender/data/100k/'  # تغيير المسار هنا

# # تحميل البيانات مع معالجة الأخطاء المحتملة
# try:
#     ratings_data = pd.read_csv(dir + 'u.data', sep='\t', names=cols_dict['ratings'])
#     users_data = pd.read_csv(dir + 'u.user', sep='|', names=cols_dict['users'])
#     items_data = pd.read_csv(dir + 'u.item', sep='|', names=cols_dict['items'] + ['other_details'], encoding='latin-1')
    
#     # تنظيف بيانات الأفلام (قد تختلف حسب هيكل ملف u.item)
#     items_data = items_data[cols_dict['items']]  # الاحتفاظ بالأعمدة المطلوبة فقط
    
#     print("تم تحميل البيانات بنجاح!")
#     print(f"أبعاد البيانات: {ratings_data.shape} (تقييمات), {users_data.shape} (مستخدمين), {items_data.shape} (أفلام)")
    
# except Exception as e:
#     print(f"حدث خطأ أثناء تحميل البيانات: {str(e)}")
#     # تحميل بيانات مثاليه في حالة الخطأ (اختياري)
#     # ratings_data, users_data, items_data = load_sample_data()

# # عرض عينة من البيانات
# display(ratings_data.head(3))
# display(users_data.head(3))
# display(items_data.head(3))

# # معلومات إضافية عن البيانات
# print("\nمعلومات إضافية:")
# print(f"عدد المستخدمين الفريدين: {ratings_data['user_id'].nunique()}")
# print(f"عدد الأفلام الفريدة: {ratings_data['movie_id'].nunique()}")
# print(f"النطاق الزمني للتقييمات: من {pd.to_datetime(ratings_data['timestamp'], unit='s').min()} إلى {pd.to_datetime(ratings_data['timestamp'], unit='s').max()}")

In [19]:

# نسخة احتياطية
users_data_og = users_data.copy(deep=True)
items_data_og = items_data.copy(deep=True)
ratings_data_og = ratings_data.copy(deep=True)
print(users_data.columns)
print(items_data.columns)
print(ratings_data.columns)

Index(['user_id', 'gender', 'age', 'occupation', 'zip_code'], dtype='object')
Index(['movie_id', 'title', 'genre'], dtype='object')
Index(['user_id', 'movie_id', 'rating', 'timestamp'], dtype='object')


In [20]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from typing import Literal, Union
import matplotlib.pyplot as plt

from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity

class Utils:
    @staticmethod
    def extract_year(items_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Extracts:
            - Year from title

        returns: Dataframes with extracted features
        '''
        # Extract year from title
        items_df['year'] = items_df['title'].str.extract(r'\((\d{4})\)').astype(int)

        return items_df

    @staticmethod
    def extract_category_avg_ratings(users_df: pd.DataFrame, items_df: pd.DataFrame, ratings_df: pd.DataFrame, k=0.6) -> pd.DataFrame:
        '''
        Extracts penalized average ratings for each category for each user using exponential decay penalty.

        users_df: DataFrame containing user information.
        items_df: DataFrame containing item information with categories as binary columns.
        ratings_df: DataFrame containing user-item interactions and ratings.
        k: Control factor for penalty steepness. Default is 0.6.

        Returns a DataFrame with users and their penalized average ratings per category.
        '''
        # Create a copy of users_df to store features
        features_df = users_df.copy()

        # Define exponential penalty function
        def exp_penalty(n, k=0.6):
            return 1 / np.exp(k * n)

        # Iterate over each category in the items_df (excluding non-categorical columns)
        for category in items_df.columns[2:]:
            # Get item IDs in the current category
            category_items = items_df[items_df[category] == 1]['movie_id']

            # Filter ratings_df to include only ratings for items in the current category
            category_ratings = ratings_df[ratings_df['movie_id'].isin(category_items)]

            # Group by user_id and calculate the average rating and count of ratings for the current category
            user_stats = category_ratings.groupby('user_id')['rating'].agg(['mean', 'count']).reset_index()
            user_stats.columns = ['user_id', f'user_avg_rating_{category}', f'count_rating_{category}']

            # Merge the user stats into features_df
            features_df = pd.merge(features_df, user_stats, on='user_id', how='left')

            # Apply exponential penalty and calculate the penalized average rating for each user
            features_df[f'user_avg_rating_{category}'] = (
                (1 - exp_penalty(features_df[f'count_rating_{category}'], k)) * features_df[f'user_avg_rating_{category}']
            )

            # Fill missing values with 0
            features_df[f'user_avg_rating_{category}'] = features_df[f'user_avg_rating_{category}'].fillna(0)

        # Select only the relevant columns
        cols = features_df.columns[:32].tolist() + [col for col in features_df.columns if col.startswith('user_avg_rating_')]
        result_df = features_df[cols]

        return result_df

    @staticmethod
    def extract_category_freq(users_df: pd.DataFrame, items_df: pd.DataFrame, ratings_df: pd.DataFrame) -> pd.DataFrame:
        # Copy users_df to avoid modifying the original dataframe
        freq_df = users_df.copy()

        # Get total interactions for each user
        total_interactions = ratings_df.groupby('user_id').size().reset_index(name='total_interactions')
        freq_df = pd.merge(freq_df, total_interactions, on='user_id', how='left').fillna(0)

        # Iterate over each category in the items_df (assuming categories are from the 3rd column onward)
        for category in items_df.columns[2:]:
            # Get movie_ids that belong to the current category
            movie_ids_in_category = items_df[items_df[category] == 1]['movie_id']

            # Count interactions in the current category for each user
            category_interactions = ratings_df[ratings_df['movie_id'].isin(movie_ids_in_category)].groupby('user_id').size().reset_index(name=f'{category}_count')

            # Merge category_interactions with freq_df
            freq_df = pd.merge(freq_df, category_interactions, on='user_id', how='left').fillna(0)

            # Calculate frequency of interactions for the current category
            freq_df[f'freq_{category}'] = freq_df[f'{category}_count'] / freq_df['total_interactions']

            # Drop the intermediate category count column
            freq_df.drop(columns=[f'{category}_count'], inplace=True)

        return freq_df.fillna(0).drop(columns=['total_interactions'])

    @staticmethod
    def extend_users_items(users_df: pd.DataFrame, items_df: pd.DataFrame, ratings_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
        '''
        Extends users and items dataframes to match the ratings dataframe
        '''
        # Extend users dataframe
        users_df = pd.merge(users_df, ratings_df[['user_id']], on='user_id', how='right')

        # Extend items dataframe
        items_df = pd.merge(items_df, ratings_df[['movie_id']], on='movie_id', how='right')

        return users_df, items_df

    @staticmethod
    def multi_hot_encode(df: pd.DataFrame, col: str, delimiter='|') -> pd.DataFrame:
        '''
        Multi hot encodes columns in a dataframe
        '''
        df_ = df.copy(deep=True)

        # Change Children's to Children to match the other genres
        df_[col] = df_[col].str.replace("Children's", 'Children') if col == 'genre' else df_[col]

        # split genres
        df_[col] = df_[col].str.split(delimiter)

        # Create a pivot table
        pivot_df = df_.explode(col).pivot_table(index='movie_id', columns=col, aggfunc='size', fill_value=0).reset_index()

        # Merge the pivot table with the original DataFrame on 'movie_id'
        result = pd.merge(df, pivot_df, on='movie_id', how='left')

        return result.drop(columns=[col])

    @staticmethod
    def one_hot_encode(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
        '''
        One hot encodes columns in a dataframe
        '''
        return pd.get_dummies(df, columns=cols) * 1

    @staticmethod
    def move_column(df: pd.DataFrame, col: list[str], pos: int) -> pd.DataFrame:
        '''
        Moves a column to a specific position in a DataFrame
        '''
        cols = df.columns.tolist()
        for i in reversed(col):
            cols.insert(pos, cols.pop(cols.index(i)))
        return df[cols]

    @staticmethod
    def preprocess_user(user: dict, num_items: int, users: np.ndarray, weights: list[np.ndarray]=None, topk: int=3, verbose=False) -> tuple[torch.IntTensor, torch.FloatTensor, Union[list[np.ndarray], None], Union[np.ndarray, None]]:
        '''
        Preprocesses user data for model input
        '''
        if 'age' not in user or not user['age']:
            user_ = users[user['id'] - 1]
            user_ = np.insert(user_, 0, user['id'])
            print(f"User id: {user['id']} top {topk} genres: {np.array(genre)[np.argsort(user_[-18:])[-topk:][::-1]]}") if verbose else None
            user_ = np.tile(user_, (num_items, 1))
            return torch.IntTensor(user_[:, 0]), torch.FloatTensor(user_[:, 1:]), None, np.array(genre)[np.argsort(user_[0, -18:])[-topk:][::-1]]

        user_ = np.zeros(31, dtype=float)

        user_[0] = user['id']

        user_[1 if user['gender'] == 'M' else 2] = 1

        user_[3 + occupation.index(user['occupation'])] = 1

        # map age to bins
        user['age'] = 1 if user['age'] < 18 else 18 if user['age'] < 25 else 25 if user['age'] < 35 else 35 if user['age'] < 45 else 45 if user['age'] < 56 else 56

        user_[3 + len(occupation) + age.index(user['age'])] = 1

        avg_ratings = np.zeros(len(genre), dtype=float) # 18 genres

        for genre_ in user['genres']:
            avg_ratings[genre.index(genre_)] = 1.0

        user_ = np.concatenate((user_, avg_ratings))

        # Get top 10 users ids of users with similar intrests (cosine similarity)
        similar_users_ids = cosine_similarity(user_[1:].reshape(1, -1), users).argsort()[0][-10:]

        # Get the mean embeddings of the top 10 similar users
        mlp_weights = weights[0][similar_users_ids].mean(axis=0)
        mf_weights = weights[1][similar_users_ids].mean(axis=0)

        user_ = np.tile(user_, (num_items, 1))
        return torch.IntTensor(user_[:, 0]), torch.FloatTensor(user_[:, 1:]), [mlp_weights, mf_weights], None

    @staticmethod
    def preprocess_items(items: pd.DataFrame) -> pd.DataFrame:
        '''
        Preprocesses items data for model input
        '''
        # multi hot encode genres
        items_ = Utils.multi_hot_encode(items, 'genre')
        items_ = Utils.extract_year(items_)
        items_['year'] = items_['year'] / items_['year'].max()
        items_ = items_.drop(['title'], axis=1)

        return items_

    @staticmethod
    def remove_missing_values(ratings: pd.DataFrame, items: pd.DataFrame) -> tuple[pd.DataFrame]:
        '''
        Removes rows with missing values from items and ratings dataframes
        '''
        # Get item ids with missing release dates
        nan_item_ids = items[items[['release_date']].isna().any(axis=1)]['item_id']

        # Remove movies with missing release dates
        items.dropna(subset=['release_date'], inplace=True)

        # remove ratings of missing items
        ratings = ratings[~ratings['item_id'].isin(nan_item_ids)]

        return ratings, items

    @staticmethod
    def negative_sampling(ratings: pd.DataFrame, items: pd.DataFrame, num_negatives: int) -> pd.DataFrame:
        '''
        Sample negative items for each user
        '''
        # All Movie ids
        all_items = items['movie_id'].values

        negative_samples = []
        for user_id in ratings['user_id'].unique():
            # Movie ids that the user has interacted with
            pos_items = ratings[ratings['user_id'] == user_id]['movie_id'].values

            # Movie ids that the user has not interacted with
            unrated_items = np.setdiff1d(all_items, pos_items)

            # Sample negative items
            neg_items = np.random.choice(unrated_items, size=num_negatives, replace=False)

            # Create negative samples
            for item_id in neg_items:
                negative_samples.append([user_id, item_id, 0])

        negative_samples = pd.DataFrame(negative_samples, columns=['user_id', 'movie_id', 'rating'])

        ratings['rating'] = [1] * ratings.shape[0]

        return pd.concat([ratings, negative_samples], ignore_index=True)

    @staticmethod
    def ndcg_hit_ratio(y_preds, X_test_users, y_true, k=10):  # تغيير هنا: قيمة افتراضية لـ k
        '''
        Compute NDCG and Hit Ratio
        '''
        if k is None:  # إضافة تحقق
            k = 10

        unique_users = np.unique(X_test_users, axis=0)
        hits = 0
        total_users = len(unique_users)

        y_preds_padded = []
        y_true_padded = []
        for user in unique_users:
            user_indices = np.where((X_test_users == user).all(axis=1))[0]
            user_preds = y_preds[user_indices][:k].flatten()
            user_true = y_true[user_indices][:k].flatten()

            if np.any(user_true == 1):
                hits += 1

            if len(user_preds) < k:
                user_preds = np.pad(user_preds, (0, k - len(user_preds)), mode='constant', constant_values=-1e10)
            if len(user_true) < k:
                user_true = np.pad(user_true, (0, k - len(user_true)), mode='constant', constant_values=0)

            y_preds_padded.append(user_preds)
            y_true_padded.append(user_true)

        ndcg = ndcg_score(y_true_padded, y_preds_padded, k=k)
        hit_ratio = hits / total_users
        return ndcg, hit_ratio



    @staticmethod
    def pipeline(request: any, model: nn.Module, weights: list[np.ndarray], users: np.ndarray, movies: pd.DataFrame, movies_og: pd.DataFrame, ratings: pd.DataFrame, mode: str) -> tuple[list[dict], Union[np.ndarray, None]]:
        '''
        Pipeline for inference
        '''
        num_items = 300 # Number of items to retrieve
        request = request if isinstance(request, dict) else request.model_dump()

        # preprocess the old user
        user_id, user, weights, top_n_genres = Utils.preprocess_user(
                                        user=request,
                                        num_items=num_items,
                                        users=users,
                                        weights=weights
                                        )
        user_id, user = user_id.to(model.device), user.to(model.device)

        movies = Utils.retrieve(
            movies=movies,
            user=user.detach().cpu().numpy(),
            num_genres=len(request['genres']) if request['genres'] else 3,
            k=num_items,
            random_state=0
        )

        movie_ids, movies = Utils.filter(
            movies=movies,
            ratings=ratings,
            user_id=request['id']
        )
        movie_ids, movies = movie_ids.to(model.device), movies.to(model.device)

        y_pred = model(
            user_id[:len(movies)],
            movie_ids,
            user[:len(movies)],
            movies,
            weights
        ).cpu().detach().numpy()

        movies_retrieved = movies_og[movies_og['movie_id'].isin(movie_ids.cpu().numpy())].sort_values(by='movie_id', key=lambda x: pd.Categorical(x, categories=movie_ids.cpu().numpy(), ordered=True))

        return Utils.order(y_pred, movies_retrieved, mode, top_k=request['top_k']).to_dict(orient='records'), top_n_genres

    @staticmethod
    def retrieve(movies: pd.DataFrame, user: np.ndarray, k: int, num_genres: int=3, random_state: int=42) -> pd.DataFrame:
        '''
        Retrieve top k movies based on genres based on this equation:
        ```
        num_movies_per_genre = k // (len(genres) + 1) # +1 for the most popular genre
        ```

        Example:
        If k = 100 and genres = ['Action', 'Adventure', 'Animation'], then:
        25 movies will be retrieved for each genre and 25 for the most popular genre.

        movies: DataFrame containing movie information.
        genres: List of genres to retrieve movies for.
        k: Number of movies to retrieve.

        Returns a DataFrame containing the top k movies based on the specified genres.
        '''


    @staticmethod
    def filter(movies: pd.DataFrame, ratings: pd.DataFrame, user_id: int) -> tuple[torch.IntTensor, torch.FloatTensor]:
        '''
        Filter movies that the user has not interacted with, and remove duplicates
        '''
        # Get movie ids that the user has interacted with
        user_movies = ratings[ratings['user_id'] == user_id]['movie_id'].values

        # Filter movies that the user has not interacted with
        movies = movies[~movies['movie_id'].isin(user_movies)]

        # Remove duplicates
        movies = movies.drop_duplicates(subset=['movie_id'])

        return torch.IntTensor(movies['movie_id'].values), torch.FloatTensor(movies.drop(columns=['movie_id']).values)

    @staticmethod
    def order(y_pred: np.ndarray, movies: pd.DataFrame, mode: Literal['explicit', 'implicit'], top_k=10) -> list[dict]:
        '''
        Order the predictions
        '''
        col_name= 'predicted_rating' if mode == 'explicit' else 'predicted_score'
        sorted_index = np.argsort(-y_pred, axis=0).reshape(-1).tolist()
        y_pred = y_pred[sorted_index]
        sorted_movies = movies.iloc[sorted_index]
        sorted_movies = sorted_movies.copy()
        sorted_movies[col_name] = y_pred if mode == 'implicit' else y_pred * 5
        sorted_movies.reset_index(drop=True, inplace=True)
        sorted_movies[col_name] = sorted_movies[col_name].apply(lambda x: round(x, 2))

        return sorted_movies.head(top_k)

    @staticmethod
    def plot_metrics(history: dict, title: str, figsize: tuple=(12, 4)) -> None:
        '''
        Plot the training and validation losses in one figure and the other metrics in another figure
        '''
        fig, ax = plt.subplots(1, 2, figsize=figsize)
        ax[0].plot(history['loss'], label='Train Loss')
        ax[0].plot(history['val_loss'], label='Validation Loss')
        ax[0].set_title('Training and Validation Loss')
        ax[0].set_xlabel('Epoch')
        ax[0].set_ylabel('Loss')
        ax[0].legend()

        # Metrics plot
        for metric, values in history.items():
            if metric not in ['loss', 'val_loss']:
                ax[1].plot(values, label=metric)

        ax[1].set_title('Metrics')
        ax[1].set_xlabel('Epoch')
        ax[1].set_ylabel('Value')
        ax[1].legend()
        plt.suptitle(title)
        plt.show()

class EarlyStopping:
    """
    Early stopping to stop the training when the loss does not improve after a certain number of epochs (patience).
    """
    def __init__(self, patience=3, delta=0, verbose=False, path='checkpoint.pth') -> None:
        """
        Args:
            patience (int): How many epochs to wait after the last time the validation loss improved.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            verbose (bool): If True, prints a message for each validation loss improvement.
            path (str): Path for the checkpoint to be saved to.
        """
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.path = path
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, val_loss, model: nn.Module) -> None:
        '''
        Call method
        '''
        score = -val_loss

        if not self.best_score:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}') if self.verbose else None
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model: nn.Module) -> None:
        '''
        Save the model checkpoint
        '''
        print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...') if self.verbose else None
        torch.save(model.state_dict(), self.path)
        self.best_loss = val_loss

age = [
    1, 18, 25, 35, 45, 50, 56
]

occupation = [
    'other', 'educator', 'artist', 'clerical', 'grad student',
    'customer service', 'doctor', 'executive', 'farmer', 'homemaker',
    'K-12 student', 'lawyer', 'programmer', 'retired', 'sales', 'scientist',
    'self-employed', 'engineer', 'craftsman', 'unemployed', 'writer'
]

genre = [
    'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
    'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]

cols_dict = {
    'ratings': ['user_id', 'movie_id', 'rating', 'timestamp'],
    'users': ['user_id', 'gender', 'age', 'occupation', 'zip_code'],
    'items': ['movie_id', 'title', 'genre'],
}

css = """
    <style>
        .card-container {
            display: flex;
            flex-direction: row;
            justify-content: center;
            align-items: start;
            gap: 20px;
            flex-wrap: wrap;
            margin: 20px 0;
        }

        .card {
            width: 100%;
            max-width: 300px;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 16px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            background-color: #eee;
            transition: transform 0.2s ease-in-out;
        }

        .card:hover {
            transform: scale(1.05);
        }

        .card-title {
            font-size: 1.25em;
            margin-bottom: 8px;
            color: #333;
        }

        .card-text {
            font-size: 1em;
            margin-bottom: 8px;
            color: #555;
        }

        .footer {
        position: fixed;
        left: 0;
        bottom: 0;
        width: 100%;
        background-color: rgb(45, 38, 48);
        color: #fff;
        text-align: center;
        padding: 10px;
    }
    </style>
"""

In [21]:
from pydantic import BaseModel, Field
from enum import Enum

class GenreEnum(str, Enum):
    Action = "Action"
    Adventure = "Adventure"
    Animation = "Animation"
    Childrens = "Children"
    Comedy = "Comedy"
    Crime = "Crime"
    Documentary = "Documentary"
    Drama = "Drama"
    Fantasy = "Fantasy"
    FilmNoir = "Film-Noir"
    Horror = "Horror"
    Musical = "Musical"
    Mystery = "Mystery"
    Romance = "Romance"
    SciFi = "Sci-Fi"
    Thriller = "Thriller"
    War = "War"
    Western = "Western"

class OccupationEnum(str, Enum):
    other = "other"
    educator = "educator"
    artist = "artist"
    clerical = "clerical"
    grad_student = "grad student"
    customer_service = "customer service"
    doctor = "doctor"
    executive = "executive"
    farmer = "farmer"
    homemaker = "homemaker"
    K_12_student = "K-12 student"
    lawyer = "lawyer"
    programmer = "programmer"
    retired = "retired"
    sales = "sales"
    scientist = "scientist"
    self_employed = "self-employed"
    engineer = "engineer"
    craftsman = "craftsman"
    unemployed = "unemployed"
    writer = "writer"

class Request(BaseModel):
    top_k: int = Field(10, ge=1, le=20, description="Number of recommendations")
    id: int = Field(..., ge=1, description="User\'s ID")
    age: int = Field(None, ge=1, le=99, description="User\'s age")
    occupation: OccupationEnum = Field(None, description="User\'s occupation")
    gender: str = Field(None, pattern="^(M|F)$", description="User\'s gender")
    genres: list[GenreEnum] = Field(None, min_items=3, max_items=5, description="User\'s favorite genres")

In [22]:
# One-hot encode categorical features
users_data = Utils.one_hot_encode(users_data, ['occupation', 'gender', 'age'])

# Multi-hot encode genres
items_data = Utils.multi_hot_encode(items_data, 'genre')

# Features Extraction
users_data = Utils.extract_category_freq(users_data, items_data, ratings_data)
items_data = Utils.extract_year(items_data)

# Negative Sampling for implicit feedback
ratings_data = Utils.negative_sampling(ratings_data, items_data, num_negatives=25) # Adds 20 * 6040 rows (num_negatives * num_users)

# Move gender columns to the front
users_data = Utils.move_column(users_data, ['gender_M', 'gender_F'], 0)

# Extend Users and Items data to match the number of rows in the ratings data
users_data, items_data = Utils.extend_users_items(users_data, items_data, ratings_data)

# Normalize continuous features
items_data['year'] = items_data['year'].astype(float) / items_data['year'].max()

# Drop unnecessary columns
ratings_data = ratings_data.drop(['timestamp'], axis=1) # Implicit Interactions
users_data = users_data.drop(['user_id', 'zip_code'], axis=1) # Demographics
items_data = items_data.drop(['movie_id', 'title'], axis=1)

print("Ratings shape:", ratings_data.shape, "\nUsers shape:", users_data.shape, "\nItems shape:", items_data.shape)

# A quick look at the data
display(ratings_data.head(3), users_data.head(3), items_data.head(3))

Ratings shape: (1151209, 3) 
Users shape: (1151209, 48) 
Items shape: (1151209, 19)


,user_id,movie_id,rating
0,1,1193,1
1,1,661,1
2,1,914,1


,gender_M,gender_F,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,...,freq_Fantasy,freq_Film-Noir,freq_Horror,freq_Musical,freq_Mystery,freq_Romance,freq_Sci-Fi,freq_Thriller,freq_War,freq_Western
0,0,1,0,0,0,0,0,0,0,0,...,0.056604,0.0,0.0,0.264151,0.0,0.113208,0.056604,0.056604,0.037736,0.0
1,0,1,0,0,0,0,0,0,0,0,...,0.056604,0.0,0.0,0.264151,0.0,0.113208,0.056604,0.056604,0.037736,0.0
2,0,1,0,0,0,0,0,0,0,0,...,0.056604,0.0,0.0,0.264151,0.0,0.113208,0.056604,0.056604,0.037736,0.0


,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.9875
1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.9980
2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0.9820


In [23]:
# ratings
y_ratings_train, y_ratings_test = train_test_split(ratings_data, test_size=0.2, shuffle=True, random_state=42)
y_ratings_val, y_ratings_test = train_test_split(y_ratings_test, test_size=0.5, shuffle=True, random_state=42)

# users
X_users_train, X_users_test = train_test_split(users_data, test_size=0.2, shuffle=True, random_state=42)
X_users_val, X_users_test = train_test_split(X_users_test, test_size=0.5, shuffle=True, random_state=42)

# items
X_items_train, X_items_test = train_test_split(items_data, test_size=0.2, shuffle=True, random_state=42)
X_items_val, X_items_test = train_test_split(X_items_test, test_size=0.5, shuffle=True, random_state=42)

# Convert to NumPy arrays
X_users_train, X_users_val, X_users_test = X_users_train.values, X_users_val.values, X_users_test.values
X_items_train, X_items_val, X_items_test = X_items_train.values, X_items_val.values, X_items_test.values
y_ratings_train, y_ratings_val, y_ratings_test = y_ratings_train.values, y_ratings_val.values, y_ratings_test.values

# Shape of the data
print("Ratings shape:", y_ratings_train.shape, y_ratings_val.shape, y_ratings_test.shape)
print("Users shape:", X_users_train.shape, X_users_val.shape, X_users_test.shape)
print("Items shape:", X_items_train.shape, X_items_val.shape, X_items_test.shape)

Ratings shape: (920967, 3) (115121, 3) (115121, 3)
Users shape: (920967, 48) (115121, 48) (115121, 48)
Items shape: (920967, 19) (115121, 19) (115121, 19)


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, roc_auc_score

import numpy as np
from math import sqrt
# from .utils import Utils, EarlyStopping
from typing import Literal

__model_version__ = '1.0.2'

class NCF(nn.Module):
    '''
    Main Ranking model
    '''
    def __init__(self, mode: Literal['explicit', 'implicit'], num_users=6040, num_items=3952, user_dim=48, item_dim=19, num_factors=32, criterion=None, dropout=0.1, lr=1e-3, weight_decay=1e-5, verbose=False, gpu=True):
        super(NCF, self).__init__()

        self.num_users = num_users
        self.num_items = num_items
        self.mode = mode

        # Embedding layers
        self.user_embedding_mlp = nn.Embedding(num_users+1,  num_factors)
        self.item_embedding_mlp = nn.Embedding(num_items+1,  num_factors)

        self.user_embedding_mf = nn.Embedding(num_users+1, num_factors)
        self.item_embedding_mf = nn.Embedding(num_items+1, num_factors)

        # Fully connected layers for user/item features
        self.user_features = nn.Sequential(
            nn.Linear(user_dim, num_factors*2),
            nn.ReLU(),
            nn.Linear(num_factors*2, num_factors),
            nn.ReLU()
        )

        self.item_features = nn.Sequential(
            nn.Linear(item_dim, num_factors*2),
            nn.ReLU(),
            nn.Linear(num_factors*2, num_factors),
            nn.ReLU()
        )

        # MLP
        self.MLP = nn.Sequential(
            nn.Linear(4 * num_factors, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_factors),
            nn.ReLU()
        )

        # NeuMF layer
        self.neu_mf = nn.Linear(2 * num_factors, 1)

        self.sigmoid = nn.Sigmoid()

        self.criterion = criterion
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay) # weight_decay is L2 regularization
        self.device = torch.device('cuda' if torch.cuda.is_available() and gpu else 'cpu')

        self.__init_weights()

        self.to(self.device)

        # Print the model architecture
        print(self, '\nRunning on: ', self.device) if verbose else None
        print(f'Number of parameters: {self.params_count():,}') if verbose else None

    def __init_data(self, input, y=None):
        '''
        Initialize the data
        '''
        X_user_id, X_item_id, X_user, X_item = input

        X_user = torch.FloatTensor(X_user).to(self.device, non_blocking=True)
        X_item = torch.FloatTensor(X_item).to(self.device, non_blocking=True)
        X_user_id = torch.IntTensor(X_user_id).to(self.device, non_blocking=True)
        X_item_id = torch.IntTensor(X_item_id).to(self.device, non_blocking=True)
        y = torch.FloatTensor(y).to(self.device, non_blocking=True) if y is not None else None


        return X_user_id, X_item_id, X_user, X_item, y

    def __init_weights(self) -> None:
        '''
        Initialize the weights
        '''
        for m in self.modules():
            if isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, mean=0, std=0.01) # Normal initialization with mean 0 and standard deviation 0.01

        for m in self.MLP.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu') # He initialization, fan_in preserves the magnitude of the variance of the weights in the forward pass
                m.bias.data.fill_(0.01)

        nn.init.xavier_uniform_(self.neu_mf.weight, gain=nn.init.calculate_gain('sigmoid')) # Glorot initialization

    def forward(self, user_id: torch.IntTensor, item_id: torch.IntTensor, user_features: torch.FloatTensor, item_features: torch.FloatTensor, weights=None)-> torch.Tensor:
        '''
        Forward pass
        '''
        # Embedding Layers
        if not weights:
            user_embedding_mlp = self.user_embedding_mlp(user_id) # (batch_size, num_factors)
            user_embedding_mf = self.user_embedding_mf(user_id) # (batch_size, num_factors)
        else: # Explicit embeddings for OOV users
            user_embedding_mlp = torch.tensor(weights[0], device=self.device).repeat(len(user_id), 1) # (batch_size, num_factors)
            user_embedding_mf = torch.tensor(weights[1], device=self.device).repeat(len(user_id), 1) # (batch_size, num_factors)

        item_embedding_mlp = self.item_embedding_mlp(item_id) # (batch_size, num_factors)
        item_embedding_mf = self.item_embedding_mf(item_id) # (batch_size, num_factors)

        # User and Item Features
        user_features = self.user_features(user_features) # (batch_size, num_factors)
        item_features = self.item_features(item_features) # (batch_size, num_factors)

        # Concatenate the embeddings and features
        user_embedding_mlp = torch.cat([user_embedding_mlp, user_features], dim=-1) # (batch_size, 2 * num_factors)
        item_embedding_mlp = torch.cat([item_embedding_mlp, item_features], dim=-1) # (batch_size, 2 * num_factors)

        # MLP Branch
        mlp_input = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1) # (batch_size, 4 * num_factors)
        mlp_output = self.MLP(mlp_input) # (batch_size, num_factors)

        # GMF Branch
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf) # (batch_size, num_factors)

        # NeuMF Layer
        neu_mf_input = torch.cat([mlp_output, mf_output], dim=-1) # (batch_size, 2 * num_factors)
        neu_mf = self.neu_mf(neu_mf_input) # (batch_size, 1)

        return self.sigmoid(neu_mf).flatten()
    def get_user_embedding(self, user_id):
        self.eval()  
        with torch.no_grad():
            user_id_tensor = torch.tensor([user_id], dtype=torch.long, device=self.device)
            user_embedding = self.user_embedding_mf(user_id_tensor)
            return user_embedding.squeeze(0).detach().cpu().numpy()
            
    def get_item_embedding(self, item_id):
        self.eval()
        with torch.no_grad():
            item_id_tensor = torch.tensor([item_id], dtype=torch.long, device=self.device)
            item_embedding = self.item_embedding_mf(item_id_tensor)
            return item_embedding.squeeze(0).detach().cpu().numpy()

    def fit(self, X: list[np.ndarray], y: np.ndarray, epochs: int, batch_size: int, X_val: list[np.ndarray] = None, y_val: np.ndarray = None, k:int = 10, scheduler: torch.optim.lr_scheduler = None, early_stopping: EarlyStopping = None) -> dict:
        '''
        Train the model

        params:
        X: list[np.ndarray] - [user_ids, item_ids, user_features, item_features]
        y: np.ndarray - Target values
        epochs: int - Number of epochs
        batch_size: int - Batch size
        k: int - Number of top-k items to consider
        X_val: list[np.ndarray] - List of users and items features for validation
        y_val: np.ndarray - Target values for validation

        returns:
        list[float] - Loss values for plotting
        '''
        X_user_id, X_item_id, X_user, X_item, y = self.__init_data(X, y)

        dataset = TensorDataset(X_user_id, X_item_id, X_user, X_item, y)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.train()

        # self.__init_weights() # Reinitialize the weights to prevent using the weights from the previous training session

        losses = []
        all_metrics = []
        lrs = []
        for epoch in range(epochs):
            total_loss = 0.0
            for i, batch in enumerate(dataloader):
                X_user_id, X_item_id, X_user, X_item, y = batch

                self.optimizer.zero_grad() # Zero the gradients
                output = self(X_user_id, X_item_id, X_user, X_item) # Forward pass
                loss = self.criterion(output, y) # Compute the loss
                loss.backward() # Backward pass
                self.optimizer.step() # Update the weights

                total_loss += loss.item()

                print(f'Epoch {epoch+1}/{epochs}') if i == 0 else None
                print(f'{i+1}/{len(dataloader)} - loss: {(total_loss / (i+1)):.4f}', end='\r' if i + 1 < len(dataloader) else ' ')

            losses.append(total_loss / (i + 1))
            # Evaluate the model
            if X_val is not None and y_val is not None:
                self.eval()
                metrics = self.evaluate(X_val, y_val, batch_size, k)

                if scheduler:
                    scheduler.step(metrics[0])
                    lrs.append(scheduler.get_last_lr()[0])

                if self.mode == 'explicit':
                    all_metrics.append([*metrics])
                    print(f'- Val Loss: {metrics[0]:.4f} - R2: {metrics[1]:.4f} - MAE: {metrics[2]:.4f} - MSE: {metrics[3]:.4f} - RMSE: {metrics[4]:.4f} - lr: {lrs[-1]}')
                else:
                    all_metrics.append([*metrics])
                    print(f'- Val Loss: {metrics[0]:.4f} - NDCG: {metrics[1]:.4f} - HR: {metrics[2]:.4f} - ROC-AUC: {metrics[3]:.4f} - lr: {lrs[-1]}')

                if early_stopping:
                    early_stopping(metrics[0], self)
                    if early_stopping.early_stop:
                        print("Early stopping")
                        break

                self.train()
            else:
                print()

        if self.mode == 'explicit':
            history = {
                'loss': losses,
                'val_loss': [m[0] for m in all_metrics],
                'r2': [m[1] for m in all_metrics],
                'mae': [m[2] for m in all_metrics],
                'mse': [m[3] for m in all_metrics],
                'rmse': [m[4] for m in all_metrics],
                'lr': lrs
            }
        else:
            history = {
                'loss': losses,
                'val_loss': [m[0] for m in all_metrics],
                'ndcg': [m[1] for m in all_metrics],
                'hr': [m[2] for m in all_metrics],
                'roc_auc': [m[3] for m in all_metrics],
                'lr': lrs
            }

        return history

    def predict(self, user_id: torch.IntTensor, item_id:torch.IntTensor, user: torch.FloatTensor, items: torch.FloatTensor)-> torch.Tensor:
        '''
        Alias for forward method, Initializes data first then calls forward method
        '''
        user_id, item_id, user_input, item_input, _ = self.__init_data([user_id, item_id, user, items])
        return self(user_id, item_id, user_input, item_input)

    def evaluate(self, X_val: list, y_val: np.ndarray, batch_size: int, k: int = None) -> tuple[float]:
        '''
        Evaluate the model

        params:
        X_val: list - List of users and items features
        y_val: np.ndarray - Target values
        batch_size: int - Batch size
        type: Literal['explicit', 'implicit'] - Type of model (explicit or implicit)
        k: int - Number of top-k items to consider
        thresh: float - Threshold for hit rate

        returns:
        tuple[float] - Val loss, MAE, R2, RMSE, MSE for explicit model
        tuple[float] - Val loss, NDCG, HR, AUC for implicit model
        '''
        user_id, item_id, user_input, item_input, target = self.__init_data(X_val, y_val)

        dataset = TensorDataset(user_id, item_id, user_input, item_input, target)
        dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=0)

        avg_loss, y_preds = 0.0, []
        with torch.no_grad():
            for i, batch in enumerate(dataloader):
                user_id, item_id, user_input, item_input, y = batch

                # Predictions
                preds = self(user_id, item_id, user_input, item_input)

                # Append to y_preds
                y_preds.extend(preds.cpu().numpy())

                # Compute loss
                loss = self.criterion(preds, y)
                avg_loss += loss.item()

        avg_loss /= (i + 1)

        target = target.cpu().numpy()
        y_preds = np.array(y_preds).reshape(-1)

        # Compute additional evaluation metrics
        if self.mode == 'explicit':
            r2 = r2_score(target, y_preds)
            mae = mean_absolute_error(target, y_preds)
            mse = mean_squared_error(target, y_preds)
            rmse = sqrt(mse)

            return avg_loss, r2, mae, mse, rmse
        else:
            ndcg, hr = Utils.ndcg_hit_ratio(y_preds, X_val[2], y_val, k)
            roc_auc = roc_auc_score(target, y_preds)

            return avg_loss, ndcg, hr, roc_auc

    def params_count(self)-> int:
        '''
        Count the number of parameters in the model
        '''
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def save_weights(self, path)-> None:
        '''
        Save the model weights
        '''
        torch.save(self.state_dict(), path)

    def load_weights(self, path, eval=True)-> None:
        '''
        Load the model weights

        Note: Set eval to True if you want to set the model to evaluation mode (dropout layers will be disabled)
        '''
        self.load_state_dict(torch.load(path, map_location=self.device))
        self.eval() if eval else None

In [25]:

user_dim = users_data.shape[1] # 48
item_dim = items_data.shape[1] # 19
num_users = int(users_data_og['user_id'].max()) + 1  # +1 لأن الفهرس يبدأ من 0
num_items = int(items_data_og['movie_id'].max()) + 1

# num_users = users_data_og['user_id'].max()
# num_items = items_data_og['movie_id'].max()

embedding_dim = 32 # number of latent factors

model = NCF(
    num_users=num_users,
    num_items=num_items,
    user_dim=user_dim,
    item_dim=item_dim,
    num_factors=embedding_dim,
    mode='implicit',
    criterion=torch.nn.BCELoss(), # BCELoss since output layer has sigmoid applied to it, otherwise BCEWithLogitsLoss()
    dropout=0.1,
    lr=1e-3,
    weight_decay=1e-5, # L2 regularization
    verbose=True,
    gpu=True
)

early_stopping = EarlyStopping(patience=2, delta=0.001, path='C:/Users/lenovo/Desktop/LinUCB-HybridRecommender/implicit.pth') # early stops after 2 consecutive epochs with minor loss decrease/increase
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model.optimizer, mode='min', factor=0.1, patience=0) # reduces learning rate by factor of 0.1 when no improvement is seen

NCF(
  (user_embedding_mlp): Embedding(6042, 32)
  (item_embedding_mlp): Embedding(3954, 32)
  (user_embedding_mf): Embedding(6042, 32)
  (item_embedding_mf): Embedding(3954, 32)
  (user_features): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (item_features): Sequential(
    (0): Linear(in_features=19, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (MLP): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=128, out_features=32, bias=True)
    (10): ReLU

In [26]:
import pandas as pd
from scipy.sparse import csr_matrix
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

def generate_association_rules(user_item_matrix, min_support=0.01, min_confidence=0.2, max_len=2):
    """
    توليد قواعد ارتباطية من مصفوفة تفاعلات المستخدمين والعناصر
    
    Args:
        user_item_matrix: مصفوفة تفاعلات (مستخدمين × عناصر) - يجب أن تكون pandas.DataFrame
        min_support: الحد الأدنى للدعم
        min_confidence: الحد الأدنى للثقة
        max_len: الطول الأقصى للقاعدة
    
    Returns:
        list: قواعد الارتباط [(antecedents, consequents, confidence), ...]
    """
    # تحويل المصفوفة إلى DataFrame إذا لم تكن كذلك
    if not isinstance(user_item_matrix, pd.DataFrame):
        user_item_matrix = pd.DataFrame.sparse.from_spmatrix(user_item_matrix)
    
    # تحويل القيم إلى بوليان (True/False)
    binary_matrix = user_item_matrix.astype(bool)
    
    # إيجاد العناصر المتكررة
    frequent_itemsets = apriori(binary_matrix, 
                              min_support=min_support, 
                              use_colnames=True,
                              max_len=max_len)
    
    # توليد القواعد الترابطية
    rules = association_rules(frequent_itemsets, 
                            metric="confidence", 
                            min_threshold=min_confidence)
    
    # تحويل النتائج إلى الشكل المطلوب
    formatted_rules = []
    for _, rule in rules.iterrows():
        antecedents = list(rule['antecedents'])
        consequents = list(rule['consequents'])
        confidence = rule['confidence']
        formatted_rules.append((antecedents, consequents, confidence))
    
    return formatted_rules

# تعديل دالة إنشاء المصفوفة لتعيد DataFrame بدلاً من csr_matrix
def create_user_item_matrix(ratings_df, num_users, num_items):
    """
    إنشاء مصفوفة تفاعلات مستخدمين-عناصر كـ DataFrame
    """
    sparse_matrix = csr_matrix((ratings_df['rating'], 
                             (ratings_df['user_id'], ratings_df['movie_id'])),
                            shape=(num_users, num_items))
    
    return pd.DataFrame.sparse.from_spmatrix(sparse_matrix)

user_item_matrix = create_user_item_matrix(ratings_data, num_users, num_items)
rules = generate_association_rules(user_item_matrix, min_support=0.05, min_confidence=0.4, max_len=2)
print(f"Generated {len(rules)} association rules")


Generated 68874 association rules


In [27]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

ncf_construction_params = {
    'num_users': int(users_data_og['user_id'].max()) + 1,
    'num_items': int(items_data_og['movie_id'].max()) + 1,
    'user_dim': users_data.shape[1],  # 48
    'item_dim': items_data.shape[1],  # 19
    'num_factors': 32,  # embedding_dim
    'mode': 'implicit',
    'criterion': torch.nn.BCELoss(),
    'dropout': 0.1,
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'verbose': True,
    'gpu': True
}

# تعديل فئة ModelTrainer لإنشاء scheduler بشكل صحيح
class ModelTrainer:
    def __init__(self, model_class, model_path='saved_models'):
        self.model_class = model_class
        self.model_path = model_path
        os.makedirs(model_path, exist_ok=True)
    
    def train(self, train_data, val_data, X_users_train, X_items_train, X_users_val, X_items_val, 
              epochs=12, batch_size=2048, patience=2, delta=0.001, **model_kwargs):
        # إنشاء النموذج
        model = self.model_class(**model_kwargs)
        
        # إنشاء early stopping مع مسار الحفظ
        early_stopping = EarlyStopping(
            patience=patience,
            delta=delta,
            path=os.path.join(self.model_path, 'best_model.pth')
        )
        
        # إنشاء scheduler بعد إنشاء النموذج
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=model.optimizer,
            mode='min',
            factor=0.1,
            patience=0
        )
        
        # التدريب
        history = model.fit(
            X=[train_data[:, 0], train_data[:, 1], X_users_train, X_items_train],
            y=train_data[:, 2],
            X_val=[val_data[:, 0], val_data[:, 1], X_users_val, X_items_val],
            y_val=val_data[:, 2],
            epochs=epochs,
            batch_size=batch_size,
            early_stopping=early_stopping,
            scheduler=scheduler
        )
        
        # حفظ النموذج النهائي
        save_path = os.path.join(self.model_path, f'{self.model_class.__name__}_final.pth')
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")
        return model, history


class HybridRecommenderLR:
    def __init__(self, neu_mf_model, rules, user_item_matrix, ratings_df, movies_df):
        self.neu_mf = neu_mf_model
        self.rules = rules
        self.user_item_matrix = user_item_matrix
        self.ratings_df = ratings_df
        self.movies_df = movies_df
        self.all_item_ids = ratings_df['movie_id'].unique()
        self._prepare_knn()
        self._validate_data()
        self.lr_model = None
        self.feature_scaler = StandardScaler()
        
    def _validate_data(self):
        required_movie_cols = {'movieId', 'title', 'genres'}
        if not required_movie_cols.issubset(self.movies_df.columns):
            missing = required_movie_cols - set(self.movies_df.columns)
            raise ValueError(f"Missing movie columns: {missing}")

    def _prepare_knn(self):
        self.item_ids = np.array(list(self.ratings_df['movie_id'].unique()))
        self.item_embeddings = {}
        valid_items = []
        embeddings = []

        for item in self.item_ids:
            try:
                emb = self.neu_mf.get_item_embedding(item)
                self.item_embeddings[item] = emb
                valid_items.append(item)
                embeddings.append(emb)
            except Exception as e:
                print(f"Warning: Error getting embedding for item {item}: {str(e)}")

        if not valid_items:
            raise ValueError("No valid items for KNN initialization")

        self.item_ids = np.array(valid_items)
        embeddings = np.array(embeddings)

        if embeddings.shape[1] > 50:
            from sklearn.decomposition import PCA
            pca = PCA(n_components=min(32, embeddings.shape[1]))
            embeddings = pca.fit_transform(embeddings)
            print(f"Reduced dimensions from {embeddings.shape[1]} to 32 using PCA")

        self.knn_model = NearestNeighbors(n_neighbors=50, metric='cosine', algorithm='brute')
        self.knn_model.fit(embeddings)

        for i, item in enumerate(valid_items):
            self.item_embeddings[item] = embeddings[i]
    
    def prepare_training_data(self, train_interactions, sample_negatives=3):
        X = []
        y = []
        pos_interactions = set((u, i) for u, i in train_interactions[:, :2])
        
        for user_id, item_id in tqdm(train_interactions[:, :2], desc="Preparing training data"):
            features = self._extract_features(user_id, item_id)
            X.append(features)
            y.append(1)
            
            user_items = set(self.ratings_df[self.ratings_df['user_id'] == user_id]['movie_id'])
            all_items = set(self.all_item_ids)
            negative_candidates = list(all_items - user_items)
            
            for _ in range(sample_negatives):
                neg_item = np.random.choice(negative_candidates)
                features = self._extract_features(user_id, neg_item)
                X.append(features)
                y.append(0)
        
        return np.array(X), np.array(y)
    
    def _extract_features(self, user_id, item_id):
        features = []
        
        # NeuMF features
        try:
            user_tensor = torch.tensor([user_id], dtype=torch.long)
            item_tensor = torch.tensor([item_id], dtype=torch.long)
            user_features = torch.tensor([X_users_train[user_id]], dtype=torch.float32)
            item_features = torch.tensor([X_items_train[item_id]], dtype=torch.float32)
            
            with torch.no_grad():
                ncf_score = self.neu_mf(user_tensor, item_tensor, user_features, item_features).item()
            features.append(ncf_score)
        except:
            features.append(0)
        
        # Rule-based features
        rule_score = 0
        for rule in self.rules:
            antecedents, consequents, confidence = rule
            if item_id in consequents:
                rule_score += confidence
        features.append(rule_score)
        
        # KNN features
        knn_score = 0
        if item_id in self.item_embeddings:
            user_items = set(self.ratings_df[self.ratings_df['user_id'] == user_id]['movie_id'])
            similarities = []
            for ui in user_items:
                if ui in self.item_embeddings:
                    sim = 1 - np.linalg.norm(self.item_embeddings[item_id] - self.item_embeddings[ui])
                    similarities.append(sim)
            if similarities:
                knn_score = np.mean(similarities)
        features.append(knn_score)
        
        # Additional features
        item_popularity = self.ratings_df['movie_id'].value_counts().get(item_id, 0)
        features.append(item_popularity)
        
        return features
    
    def train_logistic_regression(self, X_train, y_train):
        self.lr_model = make_pipeline(
            StandardScaler(),
            LogisticRegression(class_weight='balanced', max_iter=1000)
        )
        self.lr_model.fit(X_train, y_train)
        print("Logistic Regression model trained successfully")
        print(f"Feature weights: {self.lr_model.steps[1][1].coef_}")
    
    def hybrid_recommend_lr(self, user_id, item_id=None, k=10):
        if not item_id:
            rec_items, _ = self.get_neu_mf_recommendations(user_id, k*2)
        else:
            neu_mf_items, _ = self.get_neu_mf_recommendations(user_id, k)
            rule_items = [r[0] for r in self.get_rule_based_recommendations(item_id)]
            knn_items = [r[0] for r in self.get_knn_recommendations(item_id, k//2)]
            rec_items = list(set(neu_mf_items + rule_items + knn_items))
        
        if self.lr_model is None:
            print("Warning: LR model not trained, using manual hybrid")
            return self.hybrid_recommend(user_id, item_id, k)
        
        X_pred = []
        valid_items = []
        
        for item in rec_items:
            try:
                features = self._extract_features(user_id, item)
                X_pred.append(features)
                valid_items.append(item)
            except:
                continue
        
        if not valid_items:
            return []
        
        X_pred = np.array(X_pred)
        scores = self.lr_model.predict_proba(X_pred)[:, 1]
        ranked_items = sorted(zip(valid_items, scores), key=lambda x: x[1], reverse=True)[:k]
        
        recommendations = []
        for item_id, score in ranked_items:
            try:
                movie = self.movies_df[self.movies_df['movieId'] == item_id].iloc[0]
                explanation = []
                
                if score > 0.7:
                    explanation.append(f"Highly Recommended (Confidence: {score:.2f})")
                elif score > 0.4:
                    explanation.append(f"Recommended (Confidence: {score:.2f})")
                else:
                    explanation.append(f"Suggested (Confidence: {score:.2f})")
                
                genres = movie['genres'].split('|')[:3]
                
                recommendations.append({
                    'movieId': item_id,
                    'title': movie['title'],
                    'genres': ', '.join(genres),
                    'score': round(score, 4),
                    'explanation': " - ".join(explanation)
                })
            except:
                continue
        
        return recommendations

class AdvancedEvaluator:
    @staticmethod
    def evaluate_model(recommender, test_data, train_data, k=10, model_type='hybrid_lr', sample_size=None):
        test_df = pd.DataFrame(test_data[:, :2], columns=['user_id', 'movie_id'])
        train_df = pd.DataFrame(train_data[:, :2], columns=['user_id', 'movie_id'])
        
        if sample_size and sample_size < len(test_df['user_id'].unique()):
            users_sample = np.random.choice(test_df['user_id'].unique(), size=sample_size, replace=False)
            test_df = test_df[test_df['user_id'].isin(users_sample)]
        
        train_user_items = train_df.groupby('user_id')['movie_id'].apply(set).to_dict()
        test_user_items = test_df.groupby('user_id')['movie_id'].apply(set).to_dict()
        
        metrics = {
            'precision': [], 'recall': [], 'hit': 0, 'ndcg': [],
            'map': [], 'users_processed': 0, 'diversity': [], 'novelty': []
        }
        
        item_popularity = train_df['movie_id'].value_counts().to_dict()
        max_popularity = max(item_popularity.values()) if item_popularity else 1
        
        for user_id, test_items in tqdm(test_user_items.items(), desc=f'Evaluating {model_type} (k={k})'):
            try:
                recs = AdvancedEvaluator._get_recommendations(recommender, user_id, test_items, k, model_type)
                seen_items = train_user_items.get(user_id, set())
                recs = [item for item in recs if item not in seen_items][:k]
                
                if not recs:
                    continue
                
                relevant = len(test_items & set(recs))
                metrics['precision'].append(relevant / len(recs))
                metrics['recall'].append(relevant / len(test_items) if test_items else 0)
                
                if relevant > 0:
                    metrics['hit'] += 1
                
                y_true = [1 if item in test_items else 0 for item in recs]
                if len(y_true) > 1:
                    metrics['ndcg'].append(ndcg_score([y_true], [range(len(y_true), 0, -1)], k=k))
                else:
                    metrics['ndcg'].append(1.0 if y_true[0] == 1 else 0.0)
                
                precisions = []
                hit_count = 0
                for i, item in enumerate(recs, 1):
                    if item in test_items:
                        hit_count += 1
                        precisions.append(hit_count / i)
                metrics['map'].append(np.mean(precisions) if precisions else 0.0)
                
                unique_genres = set()
                for item in recs:
                    try:
                        genres = recommender.movies_df[recommender.movies_df['movieId'] == item]['genres'].iloc[0].split('|')
                        unique_genres.update(genres)
                    except:
                        continue
                metrics['diversity'].append(len(unique_genres) / len(recs) if recs else 0)
                
                novelty_scores = []
                for item in recs:
                    popularity = item_popularity.get(item, 0) / max_popularity
                    novelty_scores.append(-np.log(popularity + 1e-9))
                metrics['novelty'].append(np.mean(novelty_scores) if novelty_scores else 0)
                
                metrics['users_processed'] += 1
                
            except Exception as e:
                print(f"Error evaluating user {user_id}: {str(e)}")
                continue
        
        return {
            'model_type': model_type,
            'k': k,
            'precision@k': np.mean(metrics['precision']) if metrics['precision'] else 0,
            'recall@k': np.mean(metrics['recall']) if metrics['recall'] else 0,
            'hit_ratio': metrics['hit'] / metrics['users_processed'] if metrics['users_processed'] > 0 else 0,
            'ndcg@k': np.mean(metrics['ndcg']) if metrics['ndcg'] else 0,
            'map@k': np.mean(metrics['map']) if metrics['map'] else 0,
            'diversity': np.mean(metrics['diversity']) if metrics['diversity'] else 0,
            'novelty': np.mean(metrics['novelty']) if metrics['novelty'] else 0,
            'users_evaluated': metrics['users_processed']
        }
    
    @staticmethod
    def _get_recommendations(recommender, user_id, test_items, k, model_type):
        if model_type == 'hybrid_lr':
            first_item = next(iter(test_items)) if test_items else None
            return [r['movieId'] for r in recommender.hybrid_recommend_lr(user_id=int(user_id), item_id=first_item, k=k)]
        elif model_type == 'hybrid':
            first_item = next(iter(test_items)) if test_items else None
            return [r['movieId'] for r in recommender.hybrid_recommend(user_id=int(user_id), item_id=first_item, k=k)]
        elif model_type == 'neumf':
            rec_items, _ = recommender.get_neu_mf_recommendations(int(user_id), k)
            return rec_items
        elif model_type == 'rule':
            if test_items:
                first_item = next(iter(test_items))
                rule_recs = recommender.get_rule_based_recommendations(first_item)
                return [r[0] for r in rule_recs][:k]
            return []
        elif model_type == 'knn':
            if test_items:
                first_item = next(iter(test_items))
                knn_recs = recommender.get_knn_recommendations(first_item, k//2)
                return [r[0] for r in knn_recs][:k]
            return []
        return []



In [31]:
ncf_construction_params = {
    'num_users': int(users_data_og['user_id'].max()) + 1,
    'num_items': int(items_data_og['movie_id'].max()) + 1,
    'user_dim': users_data.shape[1],  # 48
    'item_dim': items_data.shape[1],  # 19
    'num_factors': 32,  # embedding_dim
    'mode': 'implicit',
    'criterion': torch.nn.BCELoss(),
    'dropout': 0.1,
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'verbose': True,
    'gpu': True
}

# ثانياً: تعريف EarlyStopping و Scheduler
early_stopping = EarlyStopping(patience=2, delta=0.001, path='implicit.pth')
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model.optimizer,
                                                       mode='min', factor=0.1, patience=0) # reduces learning rate by factor of 0.1 when no improvement is seen

# ثالثاً: التنفيذ الرئيسي
if __name__ == "__main__":
    # 1. تدريب نموذج NeuMF
    trainer = ModelTrainer(NCF)
    trained_model, history = trainer.train(
        train_data=y_ratings_train,
        val_data=y_ratings_val,
        X_users_train=X_users_train,
        X_items_train=X_items_train,
        X_users_val=X_users_val,
        X_items_val=X_items_val,
        epochs=1,
        batch_size=2048,
        patience=2,
        delta=0.001,
        **ncf_construction_params
    )
    
    movies_df = items_data_og[['movie_id', 'title', 'genre']].copy()
    movies_df.rename(columns={'movie_id': 'movieId', 'genre': 'genres'}, inplace=True)
    
    # 2. إنشاء وتدريب النموذج الهجين
    hybrid_rec_lr = HybridRecommenderLR(
        neu_mf_model=trained_model,
        rules=rules,
        user_item_matrix=user_item_matrix,
        ratings_df=ratings_data,
        movies_df=movies_df
    )
    
    X_train, y_train = hybrid_rec_lr.prepare_training_data(y_ratings_train)
    hybrid_rec_lr.train_logistic_regression(X_train, y_train)
    
    # 3. التقييم المتقدم
    advanced_results = run_advanced_evaluation(
        hybrid_rec=hybrid_rec_lr,
        y_ratings_train=y_ratings_train,
        y_ratings_test=y_ratings_test,
        k_values=[5, 10, 20],
        sample_size=2000
    )
    
    # 4. عرض النتائج
    print("\nFinal Results Summary:")
    print(advanced_results[['model_type', 'k', 'precision@k', 'recall@k', 'ndcg@k', 'map@k', 'diversity', 'novelty']])
    
    # 5. حفظ النموذج النهائي
    torch.save(hybrid_rec_lr, 'final_hybrid_model.pth')
    print("\nFinal hybrid model saved to final_hybrid_model.pth")

NCF(
  (user_embedding_mlp): Embedding(6042, 32)
  (item_embedding_mlp): Embedding(3954, 32)
  (user_embedding_mf): Embedding(6042, 32)
  (item_embedding_mf): Embedding(3954, 32)
  (user_features): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (item_features): Sequential(
    (0): Linear(in_features=19, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (MLP): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=128, out_features=32, bias=True)
    (10): ReLU

Preparing training data:   0%|          | 0/920967 [00:00<?, ?it/s]C:\Users\lenovo\AppData\Local\Temp\ipykernel_21388\2802823159.py:157: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  user_features = torch.tensor([X_users_train[user_id]], dtype=torch.float32)
Preparing training data:   1%|          | 10425/920967 [2:05:29<182:41:11,  1.38it/s]    


KeyboardInterrupt: 

In [ ]:
2